In [5]:
from IPython.display import display, Image, clear_output, HTML, Image
import time
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import requests
from bs4 import BeautifulSoup
import json

In [6]:
import time
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from jupyter_ui_poll import ui_events

event_info = {
    'type': '',
    'description': '',
    'time': -1
}

def wait_for_event(timeout=-1, interval=0.001, max_rate=20, allow_interupt=True):    
    start_wait = time.time()

    # set event info to be empty
    # as this is dict we can change entries
    # directly without using
    # the global keyword
    event_info['type'] = ""
    event_info['description'] = ""
    event_info['time'] = -1

    n_proc = int(max_rate*interval)+1
    
    with ui_events() as ui_poll:
        keep_looping = True
        while keep_looping==True:
            # process UI events
            ui_poll(n_proc)

            # end loop if we have waited more than the timeout period
            if (timeout != -1) and (time.time() > start_wait + timeout):
                keep_looping = False
                
            # end loop if event has occured
            if allow_interupt==True and event_info['description']!="":
                keep_looping = False
                
            # add pause before looping
            # to check events again
            time.sleep(interval)
    
    # return event description after wait ends
    # will be set to empty string '' if no event occured
    return event_info

# this function lets buttons 
# register events when clicked
def register_event(btn):
    # display button description in output area
    event_info['type'] = "click"
    event_info['description'] = btn.description
    event_info['time'] = time.time()
    return
    

In [7]:
def send_to_google_form(data_dict, form_url):
    ''' Helper function to upload information to a corresponding google form 
        You are not expected to follow the code within this function!
    '''
    form_id = form_url[34:90]
    view_form_url = f'https://docs.google.com/forms/d/e/{form_id}/viewform'
    post_form_url = f'https://docs.google.com/forms/d/e/{form_id}/formResponse'

    page = requests.get(view_form_url)
    content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')
    content = content.text[27:-1]
    result = json.loads(content)[1][1]
    form_dict = {}
    
    loaded_all = True
    for item in result:
        if item[1] not in data_dict:
            print(f"Form item {item[1]} not found. Data not uploaded.")
            loaded_all = False
            return False
        form_dict[f'entry.{item[4][0][0]}'] = data_dict[item[1]]
    
    post_result = requests.post(post_form_url, data=form_dict)
    return post_result.ok


In [8]:
question_1 = Image("question_1.jpg")
question_2 = Image("question_2.png")
question_3 = Image("question_3.png")
question_4 = Image("question_4.png")
question_5 = Image("question_5.png")
question_6 = Image("question_6.png")
question_7 = Image("question_7.png")
question_8 = Image("question_8.png")
question_9 = Image("question_9.png")
sr_comment = Image("sr_comment.png")

my_question_list = [question_1, question_2, question_3, question_4, question_5, question_6, question_7, question_8, question_9]
ans_n = ["b", "b", "a", "c", "d", "a", "b", "c", "d"]

q_order = [0, 1, 2, 3, 4, 5, 6, 7, 8]
t_list = []
ans_list = []
input_1= ""
input_1 = input("Enter your UCL ID: ")

input_2= ""
input_2 = input("Enter your gender: ")

input_3 = ""
input_3 = input("Enter your age: ")

print("Please read:")
print("")
print("we wish to record your response data")
print("to an anonymised public data repository. ")
print("Your data will be used for educational teaching purposes") 
print("practising data analysis and visualisation.")
print("")
print("Please type yes in the box below if you consent to the upload.")
result = input("> ")
if result == "yes":
    print("Thanks - your data will be uploaded.")
        #send_to_google_form(data_dict, form_url)
else:
    print("No problem we hope you enjoyed the test.")

print("Welcome to the Spatial test")
time.sleep(1)
    
print("Please solve the following spatial reasoning questions:")
time.sleep(1)

display(sr_comment)
time.sleep(5)

btn1 = widgets.Button(description="a")
btn2 = widgets.Button(description="b")
btn3 = widgets.Button(description="c")
btn4 = widgets.Button(description="d")

btn1.on_click(register_event) 
btn2.on_click(register_event) 
btn3.on_click(register_event)
btn4.on_click(register_event) 

panel = widgets.HBox([btn1, btn2, btn3, btn4])

start_time = time.time()

score = 0

for i in q_order:
    display(my_question_list[i])
    display (panel)
    stime=time.time()
    result = wait_for_event(timeout=22)
    etime=time.time()
    t=etime-stime
    t_list.append(t)
    ans_list.append(result['description'])
    clear_output()
    print(type(f"User clicked: {result['description']}")) 
    print(type(ans_n[i]))
    print(result['description'] == ans_n[i])
    if result['description'] == ans_n[i]:
        print("correct")
        new_score = score + 1
        score = new_score
        print(score)
    else:
        print("incorrect")
        new_score = score + 0  
        score = new_score
        print(score)
    
    
    
end_time = time.time()
time_taken = end_time - start_time


print(f"You took {time_taken:.2f} seconds")  
print (f"You're score is {score} / 9")

    
form_url= "https://docs.google.com/forms/d/e/1FAIpQLSdFngM9tsC-8kswJuKyfxllYZDuGYTNPja7WAbc23a7RuYJOA/viewform?usp=sf_link"

data_dict = {'ID': input_1, 'gender': input_2, 'age' : input_3, 'a1':ans_list[0], 't1': t_list[0], 'a2':ans_list[1], 't2':t_list[1], 'a3':ans_list[2], 't3':t_list[2], 'a4':ans_list[3],
            't4':t_list[3], 'a5':ans_list[4], 't5':t_list[4], 'a6':ans_list[5], 't6':t_list[5], 'a7':ans_list[6], 't7':t_list[6], 
            'a8':ans_list[7], 't8':t_list[7], 'a9':ans_list[8], 't9':t_list[8]}

send_to_google_form(data_dict, form_url)

<class 'str'>
<class 'str'>
True
correct
9
You took 114.51 seconds
You're score is 9 / 9


True